# Testes - Temperatura

Una vez que los datos son alistados, se realiza el análisis de testes. 

Para temperatura, se consideran los siguientes testes:

- Test de rango fijo
- Tests de consistencia interna
- Test de ceros
- Test para el salto térmico diario
- Tests de rango dinámico, consistencia temporal, persistencia y coherencia espacial

In [2]:
import pandas as pd
import numpy as np
import os
import re
from functools import reduce

In [3]:
PATH_DADOS = '../dados/'
NOMBRE_ESTACION = 'CHUCURI'
FILE_DATOS_PROCESADOS = NOMBRE_ESTACION.lower() + '_procesado.csv'

In [49]:
df = pd.read_csv(PATH_DADOS + FILE_DATOS_PROCESADOS, 
                 sep = ';', 
                 parse_dates = ["FECHA"],
                 date_parser = lambda col: pd.to_datetime(col)
                )

dfTemp = df[['FECHA', 'TEMP']].copy()
dfTemp = dfTemp.sort_values(by=['FECHA']).reset_index(drop = True)
dfTemp

,FECHA,TEMP
0,2011-01-01 01:00:00-05:00,20.6
1,2011-01-01 02:00:00-05:00,20.4
2,2011-01-01 03:00:00-05:00,20.2
3,2011-01-01 04:00:00-05:00,19.4
4,2011-01-01 05:00:00-05:00,19.2
...,...,...
266158,2017-12-31 23:10:00-05:00,NaN
266159,2017-12-31 23:20:00-05:00,NaN
266160,2017-12-31 23:30:00-05:00,NaN
266161,2017-12-31 23:40:00-05:00,NaN


In [50]:
t_max = 50.
t_min = 15.
lamb_func = lambda r: np.nan if ((r > t_max) | (r < t_min)) else r
dfTemp['TEMP'] = dfTemp['TEMP'].apply(lamb_func)
dfTemp

,FECHA,TEMP
0,2011-01-01 01:00:00-05:00,20.6
1,2011-01-01 02:00:00-05:00,20.4
2,2011-01-01 03:00:00-05:00,20.2
3,2011-01-01 04:00:00-05:00,19.4
4,2011-01-01 05:00:00-05:00,19.2
...,...,...
266158,2017-12-31 23:10:00-05:00,NaN
266159,2017-12-31 23:20:00-05:00,NaN
266160,2017-12-31 23:30:00-05:00,NaN
266161,2017-12-31 23:40:00-05:00,NaN


In [39]:
r = 30.
np.nan if ((r > t_max) | (r < t_min)) else r

30.0

In [18]:
# Reduciendo dataframes en uno solo
df = reduce(lambda df1,df2: pd.merge(df1,df2,on='FECHA', how='outer'), list_df)

# Formatando data
df['FECHA'] = pd.to_datetime(df['FECHA'], format='%Y%m%d%H:%M:%S').dt.tz_localize('America/Bogota')

In [27]:
columns = [col for col in df.columns if col != 'FECHA']

for col in columns:
    # Substituyendo valores no encontrados por np.nan
    df[col] = df[col].replace(['---[10]', '---[2]', '---[5]', '---[15]', '---'], np.NaN)
    # Transformando en formato numérico
    df[col] = pd.to_numeric(df[col], downcast="float")

In [39]:
df.to_csv(PATH_DADOS + FILE_DATOS_PROCESADOS, header=True, index=False, sep = ';')